In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import plotly.figure_factory as ff

import numpy as np
import pandas as pd
import plotly.express as px
from pandas_profiling import ProfileReport

from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)


In [ ]:
df = pd.read_csv('../input/new-york-times-covid19-dataset/us.csv')
df.head(5)

In [ ]:
profile = ProfileReport(df, title="Pandas Profiling Report - NYT Covid-19 Dataset")
profile.to_widgets()

In [ ]:
#Try different types of regression
#Importing warnings
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.linear_model import LogisticRegression,Lasso,LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn import metrics
from scipy.stats import zscore
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.gaussian_process.gpr import GaussianProcessRegressor
from  sklearn.isotonic import IsotonicRegression
from sklearn.linear_model.bayes import ARDRegression
from sklearn.linear_model.huber import HuberRegressor
from sklearn.linear_model.base import LinearRegression
from sklearn.linear_model.passive_aggressive import PassiveAggressiveRegressor 
#from sklearn.linear_model.randomized_l1 import RandomizedLogisticRegression
from sklearn.linear_model.stochastic_gradient import SGDRegressor
from sklearn.linear_model.theil_sen import TheilSenRegressor
from sklearn.linear_model.ransac import RANSACRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors.regression import KNeighborsRegressor
from sklearn.neighbors.regression import RadiusNeighborsRegressor
from sklearn.neural_network.multilayer_perceptron import MLPRegressor
from sklearn.tree.tree import DecisionTreeRegressor
from sklearn.tree.tree import ExtraTreeRegressor
from sklearn.svm.classes import SVR
from sklearn.linear_model import BayesianRidge
from sklearn.cross_decomposition import CCA
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Lars
from sklearn.linear_model import LarsCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LassoLars
from sklearn.linear_model import LassoLarsIC
from sklearn.linear_model import LassoLarsCV
from sklearn.linear_model import MultiTaskElasticNet
from sklearn.linear_model import MultiTaskElasticNetCV
from sklearn.linear_model import MultiTaskLasso
from sklearn.linear_model import MultiTaskLassoCV
from sklearn.svm import NuSVR
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import OrthogonalMatchingPursuitCV
from sklearn.cross_decomposition import PLSCanonical
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR

In [ ]:
from statsmodels.tsa.stattools import adfuller
from numpy import log


In [ ]:

X = df.drop(columns=['date','geoid', 'cases_avg', 'cases_avg_per_100k', 'deaths_avg', 'deaths_avg_per_100k'])
y = df["cases"]
y2 = df["deaths"]
sc = StandardScaler()
a = sc.fit_transform(X)
df_x = pd.DataFrame(a,columns=X.columns)
df_x.head()

result = adfuller(X.cases.dropna())
print('ADF Statistic: %f' % result[0] + ' p-value: %f' % result[1])
result = adfuller(X.deaths.dropna())
print('ADF Statistic: %f' % result[0] + ' p-value: %f' % result[1])

### Leveraged insights from [Prashant Banerjee's ARIMA Kernel](https://www.kaggle.com/prashant111/arima-model-for-time-series-forecasting)

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
model = ARIMA(df.cases, order=(1,1,2))
model_fit = model.fit(disp=0)
print(model_fit.summary())
model = ARIMA(df.deaths, order=(1,1,2))
model_fit = model.fit(disp=0)
print(model_fit.summary())
model = ARIMA(df.cases, order=(1,1,1))
model_fit = model.fit(disp=0)
print(model_fit.summary())
model = ARIMA(df.deaths, order=(1,1,1))
model_fit = model.fit(disp=0)
print(model_fit.summary())

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.figsize':(16,12), 'figure.dpi':120})
model_fit.plot_predict(dynamic=False)
plt.show()

In [ ]:
def plot_forecast(test,train,alphaval,titleval):
    model = ARIMA(train, order=(1, 1, 1))  
    fitted = model.fit(disp=-1)  
    fc, se, conf = fitted.forecast(130, alpha=alphaval) 
    fc_series = pd.Series(fc, index=test.index)
    lower_series = pd.Series(conf[:, 0], index=test.index)
    upper_series = pd.Series(conf[:, 1], index=test.index)
    plt.figure(figsize=(12,5), dpi=100)
    plt.plot(train, label='Training')
    plt.plot(test, label='Actual')
    plt.plot(fc_series, label='Forecast')
    plt.fill_between(lower_series.index, lower_series, upper_series, color='k', alpha=.30)
    plt.title(titleval)
    plt.legend(loc='lower left', fontsize=12)
    plt.show()

In [ ]:
from statsmodels.tsa.stattools import acf
train = df.deaths[:460]
test = df.deaths[460:]
plot_forecast(test,train,0.05, 'Forecast Vs Actuals for COVID-19 Deaths')
train = df.cases[:460]
test = df.cases[460:]
plot_forecast(test,train,0.05,  'Forecast Vs Actuals for COVID-19 Cases')

In [ ]:
!pip install pmdarima
from statsmodels.tsa.arima_model import ARIMA

import pmdarima as pm

model = pm.auto_arima(df.deaths, start_p=1, start_q=1,
                      test='adf',       
                      max_p=2, max_q=3, 
                      m=1,              
                      d=None,          
                      seasonal=False,   
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())

model2 = pm.auto_arima(df.cases, start_p=1, start_q=1,
                      test='adf',       
                      max_p=3, max_q=3, 
                      m=1,              
                      d=None,          
                      seasonal=False,   
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model2.summary())

In [ ]:
model.plot_diagnostics(figsize=(10,8))
plt.show()
model2.plot_diagnostics(figsize=(10,8))
plt.show()

In [ ]:
n_periods = 96
fc, confint = model.predict(n_periods=n_periods, return_conf_int=True)
index_of_fc = np.arange(len(df.deaths), len(df.deaths)+n_periods)
fc_series = pd.Series(fc, index=index_of_fc)
lower_series = pd.Series(confint[:, 0], index=index_of_fc)
upper_series = pd.Series(confint[:, 1], index=index_of_fc)
plt.plot(df.deaths)
plt.plot(fc_series, color='violet')
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.05)
plt.title("Final Forecast of COVID-19 Deaths")
plt.show()

In [ ]:
n_periods = 192
fc, confint = model2.predict(n_periods=n_periods, return_conf_int=True)
index_of_fc = np.arange(len(df.cases), len(df.cases)+n_periods)
fc_series = pd.Series(fc, index=index_of_fc)
lower_series = pd.Series(confint[:, 0], index=index_of_fc)
upper_series = pd.Series(confint[:, 1], index=index_of_fc)
plt.plot(df.cases)
plt.plot(fc_series, color='red')
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.05)
plt.title("Final Forecast of COVID-19 Deaths")
plt.show()

In [ ]:
#Splitting the data into training and testing data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df_x,y,test_size=0.20,random_state=45)
x_train2,x_test2,y_train2,y_test2=train_test_split(df_x,y2,test_size=0.20,random_state=45)

In [ ]:
#Machine Learning for Cases

model = [AdaBoostRegressor(),
         ARDRegression(),
         BayesianRidge(),
         CCA(),
         DecisionTreeRegressor(),
         ElasticNet(),
         ElasticNetCV(),
         ExtraTreeRegressor(),
         GaussianProcessRegressor(),
         GradientBoostingRegressor(),
         HuberRegressor(),
         #IsotonicRegression(), #Not feasible with the dataset
         KernelRidge(),
         KNeighborsRegressor(),
         #LabelEncoder(), #Not feasible with the dataset
         Lars(),
         LarsCV(),
         Lasso(),
         LassoCV(),
         LassoLars(),
         LassoLarsCV(),
         LassoLarsIC(),
         LinearRegression(),
         LinearSVR(),
         MLPRegressor(),
         #MultiOutputRegressor(), #Not feasible with the dataset
         #MultiTaskElasticNet(),
         #MultiTaskElasticNetCV(),
         #MultiTaskLasso(),
         #MultiTaskLassoCV(),
         OrthogonalMatchingPursuit(),
         OrthogonalMatchingPursuitCV(),
         PassiveAggressiveRegressor(),
         #PCA(), #Not feasible with the dataset
         PLSCanonical(),
         PLSRegression(),
         RadiusNeighborsRegressor(),
         #RandomizedLogisticRegression(), #Not feasible with the dataset
         RANSACRegressor(),
         Ridge(),
         RidgeCV(),
         SGDRegressor(),
         #StandardScaler(),#Not feasible with the dataset
         SVR(),
         TheilSenRegressor(),
         NuSVR()
]
model_name = []
model_score = []
model_mae = []
model_mse = []
model_rmse = []
model_r2= []

for m in model:
    #print(m)
    m.fit(x_train,y_train)
    score=m.score(x_train,y_train)
    predm=m.predict(x_test)
    model_name.append(m) 
    model_score.append(score) 
    model_mae.append(mean_absolute_error(y_test,predm))
    model_mse.append(mean_squared_error(y_test,predm))
    model_rmse.append(np.sqrt(mean_squared_error(y_test,predm)))
    model_r2.append(r2_score(y_test,predm))

list_of_models = list(zip(model_name,
                          model_score,
                          model_mae,
                          model_mse,
                          model_rmse,
                          model_r2
                         )
                     )
list_of_models
dfm = pd.DataFrame(list_of_models,columns=['Model','Score','MAE','MSE','RMSE','R2 Score'])
dfm.sort_values('Score', ascending=False)
dfm.style.background_gradient(cmap ='RdYlGn')


In [ ]:
print("*" * 40)
print("The Best Model is : ")
print("*" * 40)
print(dfm.loc[dfm['Score'].idxmax()])
print("*" * 40)

In [ ]:
print("*" * 40)
print("The Best R2 Score is : ")
print("*" * 40)
print(dfm.loc[dfm['R2 Score'].idxmax()])
print("*" * 40)

In [ ]:
#Machine LEarning for Deaths
model = [AdaBoostRegressor(),
         ARDRegression(),
         BayesianRidge(),
         CCA(),
         DecisionTreeRegressor(),
         ElasticNet(),
         ElasticNetCV(),
         ExtraTreeRegressor(),
         GaussianProcessRegressor(),
         GradientBoostingRegressor(),
         HuberRegressor(),
         #IsotonicRegression(), #Not feasible with the dataset
         KernelRidge(),
         KNeighborsRegressor(),
         #LabelEncoder(), #Not feasible with the dataset
         Lars(),
         LarsCV(),
         Lasso(),
         LassoCV(),
         LassoLars(),
         LassoLarsCV(),
         LassoLarsIC(),
         LinearRegression(),
         LinearSVR(),
         MLPRegressor(),
         #MultiOutputRegressor(), #Not feasible with the dataset
         #MultiTaskElasticNet(),
         #MultiTaskElasticNetCV(),
         #MultiTaskLasso(),
         #MultiTaskLassoCV(),
         OrthogonalMatchingPursuit(),
         OrthogonalMatchingPursuitCV(),
         PassiveAggressiveRegressor(),
         #PCA(), #Not feasible with the dataset
         PLSCanonical(),
         PLSRegression(),
         RadiusNeighborsRegressor(),
         #RandomizedLogisticRegression(), #Not feasible with the dataset
         RANSACRegressor(),
         Ridge(),
         RidgeCV(),
         SGDRegressor(),
         #StandardScaler(),#Not feasible with the dataset
         SVR(),
         TheilSenRegressor(),
         NuSVR()
]
model_name = []
model_score = []
model_mae = []
model_mse = []
model_rmse = []
model_r2= []

for m in model:
    #print(m)
    m.fit(x_train2,y_train2)
    score=m.score(x_train2,y_train2)
    predm=m.predict(x_test2)
    model_name.append(m) 
    model_score.append(score) 
    model_mae.append(mean_absolute_error(y_test2,predm))
    model_mse.append(mean_squared_error(y_test2,predm))
    model_rmse.append(np.sqrt(mean_squared_error(y_test2,predm)))
    model_r2.append(r2_score(y_test2,predm))

list_of_models = list(zip(model_name,
                          model_score,
                          model_mae,
                          model_mse,
                          model_rmse,
                          model_r2
                         )
                     )
list_of_models
dfm2 = pd.DataFrame(list_of_models,columns=['Model2','Score2','MAE2','MSE2','RMSE2','R2 Score2'])
dfm2.sort_values('Score2', ascending=False)
dfm2.style.background_gradient(cmap ='RdYlGn')

In [ ]:
print("*" * 40)
print("The Best Model for forecasting COVID-19 deaths is : ")
print("*" * 40)
print(dfm2.loc[dfm2['Score2'].idxmax()])
print("*" * 40)
print("The Best Model with best R2 Score: ")
print("*" * 40)
print(dfm2.loc[dfm2['R2 Score2'].idxmax()])
print("*" * 40)